# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [2]:
import pandas as pd
import pymongo
from pymongo import MongoClient

In [3]:
client = MongoClient("mongo")
database = client['exercices']
collection = database['youtube']

In [4]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Netoyer les données

In [135]:
collection.drop()

### Importer les données

In [136]:
l_d = df_youtube.to_dict(orient='records')
collection.insert_many(l_d)

## Question 1  

In [137]:
collection.find({"channel_title": "Apple"}).count()

11

## Question 2

In [138]:
cur = collection.aggregate([{"$group": {"_id": "$category_id"}}, {"$group": {"_id": "category_id", "count": {"$sum": 1}}}])
list(cur)

[{'_id': 'category_id', 'count': 16}]

## Question 3

## Question 4

In [139]:
cur = collection.find().sort([("views", -1)]).limit(5)
list(cur)

[{'_id': ObjectId('602e6f4e368c0c00a05b8e2f'),
  'video_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': 'BIGHIT|빅히트|방탄소년단|BTS|BANGTAN|방탄',
  'views': 41500672,
  'likes': 2010366,
  'dislikes': 78076,
  'comment_total': 736179,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': 22.09},
 {'_id': ObjectId('602e6f4e368c0c00a05b8d5a'),
  'video_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': 'BIGHIT|빅히트|방탄소년단|BTS|BANGTAN|방탄',
  'views': 38013692,
  'likes': 1950475,
  'dislikes': 75294,
  'comment_total': 713293,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': 21.09},
 {'_id': ObjectId('602e6f4e368c0c00a05b876d'),
  'video_id': 'tt2k8PGm-TI',
  'title': 'ZAYN - Dusk Till Dawn ft. Sia',
  'channel_title': 'ZaynVEVO',
  'category_id': 10,
  'tags': 'Dusk Till Dawn|Pop|RCA Records 

## Question 5

In [140]:
cur = collection.aggregate([{"$group" : {"_id" : "$category_id", "averageViewsByCategory" : {"$avg" : "$views"}}}])
list(cur)

[{'_id': 26, 'averageViewsByCategory': 537665.1806674339},
 {'_id': 15, 'averageViewsByCategory': 651404.3879310344},
 {'_id': 17, 'averageViewsByCategory': 728434.3243902439},
 {'_id': 23, 'averageViewsByCategory': 1240073.294039735},
 {'_id': 43, 'averageViewsByCategory': 8492.5},
 {'_id': 2, 'averageViewsByCategory': 607693.8706896552},
 {'_id': 22, 'averageViewsByCategory': 971532.7573696146},
 {'_id': 29, 'averageViewsByCategory': 1110334.2142857143},
 {'_id': 24, 'averageViewsByCategory': 1154868.4397251718},
 {'_id': 19, 'averageViewsByCategory': 464041.0833333333},
 {'_id': 27, 'averageViewsByCategory': 547582.4700598803},
 {'_id': 1, 'averageViewsByCategory': 1039472.6560846561},
 {'_id': 20, 'averageViewsByCategory': 681081.8170731707},
 {'_id': 10, 'averageViewsByCategory': 1176553.612},
 {'_id': 25, 'averageViewsByCategory': 540955.5569823436},
 {'_id': 28, 'averageViewsByCategory': 924730.548828125}]

## Question 6 

In [142]:
cur = collection.aggregate([{"$group": {"_id": "$channel_title", "averageLike": {"$avg": "$likes"}}}, {"$sort": {"averageLike": -1}}, {"$limit": 5}]);
list(cur)

[{'_id': 'ZaynVEVO', 'averageLike': 1431683.0},
 {'_id': 'ibighit', 'averageLike': 1371766.25},
 {'_id': 'melanie martinez', 'averageLike': 911871.5},
 {'_id': 'BETNetworks', 'averageLike': 769687.625},
 {'_id': 'jypentertainment', 'averageLike': 758826.6666666666}]